#IMPORTING DATASET

In [3]:
import pandas as pd
from pandas.api.types import is_string_dtype

d=pd.read_csv('Heart.csv')
#print(d.columns)
#checking missing values
#for i in d:
  #print(d[i].unique())
#no missing value found

#SPLIT


In [4]:
from sklearn.model_selection import train_test_split

x = d.loc[:,d.columns!='target']
y = d['target']
#print(x)
#print(y)
#print(y.unique())
train_val_x, test_x, train_val_y, test_y = train_test_split(x,y,test_size=0.2,random_state=1)
train_x, val_x, train_y, val_y = train_test_split(train_val_x,train_val_y,test_size=0.08,random_state=1)
#print(len(train_x),len(val_x),len(test_x))
#print(len(train_y),len(val_y),len(test_y))

KeyError: ignored

#Continuous Feature Handling(Binarization)

In [ ]:
from sklearn.preprocessing import Binarizer

con_col=['age','trestbps','chol','thalach','oldpeak']
b_train_x = pd.DataFrame()
b_val_x= pd.DataFrame()
b_test_x = pd.DataFrame()
b_train_val_x= pd.DataFrame()

train_x.reset_index(drop=True, inplace=True)
val_x.reset_index(drop=True, inplace=True)
test_x.reset_index(drop=True, inplace=True)
train_val_x.reset_index(drop=True, inplace=True)

for i in train_x.columns:
  if i in con_col:
    bin=Binarizer(threshold=train_x[i].mean())
    b_train_x[i]=bin.transform(train_x[[i]]).flatten()
    b_val_x[i]=bin.transform(val_x[[i]]).flatten()
    b_train_val_x[i]=bin.transform(train_val_x[[i]]).flatten()
    b_test_x[i]=bin.transform(test_x[[i]]).flatten()
  else:
    b_train_x[i] = train_x[i].copy()
    b_train_val_x[i] = train_val_x[i].copy()
    b_val_x[i] = val_x[i].copy()
    b_test_x[i] = test_x[i].copy()
#print(b_train_x)
#print(b_val_x)
#print(b_test_x)
#for i in b_train_x:
  #print(b_train_x[i].unique())

#Continuous Feature Handling(Quantization)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

q_train_x = pd.DataFrame()
q_val_x= pd.DataFrame()
q_test_x = pd.DataFrame()
q_train_val_x= pd.DataFrame()


for col in train_x.columns:
  if col in con_col:
    bin = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
    bin.fit(train_x[[col]])
    q_train_x[col] = bin.transform(train_x[[col]]).flatten()
    q_train_val_x[col] = bin.transform(train_val_x[[col]]).flatten()
    q_val_x[col] = bin.transform(val_x[[col]]).flatten()
    q_test_x[col] = bin.transform(test_x[[col]]).flatten()
  else:
    q_train_x[col] = train_x[col].copy()
    q_train_val_x[col] = train_val_x[col].copy()
    q_val_x[col] = val_x[col].copy()
    q_test_x[col] = test_x[col].copy()

#print(q_train_x)
#print(q_val_x)
#print(q_test_x)

















#Random Forest Classifier Predicting

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

n_estimatorslist=[10,20,40,60,80,100,120,140,160,180,200]
criterion_list=['gini','entropy']

#for binarization part
best_acc = -np.inf

for n in n_estimatorslist:
  for c in criterion_list:
    rfmodel=RandomForestClassifier(n_estimators=n,criterion=c,random_state=1)
    rfmodel.fit(b_train_x,train_y)
    predictions=rfmodel.predict(b_val_x)
    acc=accuracy_score(val_y, predictions)
    if acc > best_acc:
      best_acc=acc
      best_n_estimators=n
      best_criterion=c
#print(best_acc,best_n_estimators,best_criterion)

rfmodel=RandomForestClassifier(n_estimators=best_n_estimators,criterion=best_criterion,random_state=1)
rfmodel.fit(b_train_val_x,train_val_y)
predictions=rfmodel.predict(b_test_x)
acc=accuracy_score(test_y, predictions)
print(f'RandomForestBinarization Accuracy: {acc}')

#for quantization part
best_acc = -np.inf
for n in n_estimatorslist:
  for c in criterion_list:
    rfmodel=RandomForestClassifier(n_estimators=n,criterion=c,random_state=1)
    rfmodel.fit(q_train_x,train_y)
    predictions=rfmodel.predict(q_val_x)
    acc=accuracy_score(val_y, predictions)
    if acc > best_acc:
      best_acc=acc
      best_n_estimators=n
      best_criterion=c
#print(best_acc,best_n_estimators,best_criterion)
rfmodel=RandomForestClassifier(n_estimators=best_n_estimators,criterion=best_criterion,random_state=1)
rfmodel.fit(q_train_val_x,train_val_y)
predictions=rfmodel.predict(q_test_x)
acc=accuracy_score(test_y, predictions)
print(f'RandomForestQuantization Accuracy: {acc}')

# Logistic Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
c_list=[0.001, 0.01, 0.1, 1, 10, 100, 1000]
maxiter_list= list(range(100,800,100))

#for binarization part
best_acc = -np.inf

for c in c_list:
  for iter in maxiter_list:
    lgmodel=LogisticRegression(C=c,max_iter=iter,random_state=1)
    lgmodel.fit(b_train_x,train_y)
    predictions=lgmodel.predict(b_val_x)
    acc=accuracy_score(val_y, predictions)
    if acc > best_acc:
      best_acc=acc
      best_c=c
      best_iter=iter

lgmodel=LogisticRegression(C=best_c,max_iter=best_iter,random_state=1)
lgmodel.fit(b_train_val_x,train_val_y)
predictions=lgmodel.predict(b_test_x)
acc=accuracy_score(test_y, predictions)
print(f'LogisticRegressionBinarization Accuracy: {acc}')



#for quantization
best_acc = -np.inf

for c in c_list:
  for iter in maxiter_list:
    lgmodel=LogisticRegression(C=c,max_iter=iter,random_state=1)
    lgmodel.fit(q_train_x,train_y)
    predictions=lgmodel.predict(q_val_x)
    acc=accuracy_score(val_y, predictions)
    if acc > best_acc:
      best_acc=acc
      best_c=c
      best_iter=iter

lgmodel=LogisticRegression(C=best_c,max_iter=best_iter,random_state=1)
lgmodel.fit(q_train_val_x,train_val_y)
predictions=lgmodel.predict(q_test_x)
acc=accuracy_score(test_y, predictions)
print(f'LogisticRegressionQuantization Accuracy: {acc}')

#Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
alpha_list=[1.0,2.0,4.0,6.0,8.0,10.0,1.2,1.4,1.6,1.8]

#for binarization part
best_acc = -np.inf

for a in alpha_list:
  mnModel=MultinomialNB(alpha=a)
  mnModel.fit(b_train_x,train_y)
  predictions=mnModel.predict(b_val_x)
  acc=accuracy_score(val_y, predictions)
  if acc > best_acc:
    best_acc=acc
    best_alpha=a
#print(best_acc,best_alpha)

mnModel=MultinomialNB(alpha=best_alpha)
mnModel.fit(b_train_val_x,train_val_y)
predictions=mnModel.predict(b_test_x)
acc=accuracy_score(test_y, predictions)
print(f'NaiveBayesBinarization Accuracy: {acc}')

#for quantization part
best_acc = -np.inf

for a in alpha_list:
  mnModel=MultinomialNB(alpha=a)
  mnModel.fit(q_train_x,train_y)
  predictions=mnModel.predict(q_val_x)
  acc=accuracy_score(val_y, predictions)
  if acc > best_acc:
    best_acc=acc
    best_alpha=a
#print(best_acc,best_alpha)
mnModel=MultinomialNB(alpha=best_alpha)
mnModel.fit(q_train_val_x,train_val_y)
predictions=mnModel.predict(q_test_x)
acc=accuracy_score(test_y, predictions)
print(f'NaiveBayesQuantization Accuracy: {acc}')


#SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
c_list=[0.1,1.0,2.0,4.0,6, 10, 100]
gamma_list=[0.01,0.001,0.0001,0.1,0.00001,1]


#for binarization part
best_acc = -np.inf

for c in c_list:
  for gamma in gamma_list:
    svm = SVC(C=c,gamma = gamma,random_state=1)
    svm.fit(b_train_x,train_y)
    predictions=svm.predict(b_val_x)
    acc=accuracy_score(val_y, predictions)
    if acc > best_acc:
      best_acc=acc
      best_c=c
      best_gamma=gamma

#print("Binarization SVM best acc = ", best_acc)
#print("Binarization SVM best c = ", best_c)
#print("Binarization SVM best acc = ", best_gamma)

svm = SVC(C=best_c,gamma = best_gamma,random_state=1)
svm.fit(b_train_val_x,train_val_y)
predictions=svm.predict(b_test_x)
acc=accuracy_score(test_y, predictions)
print(f'SVMBinarization Accuracy: {acc}')

#for quantization part
best_acc = -np.inf
for c in c_list:
  for gamma in gamma_list:
    svm=SVC(C=c,gamma = gamma,random_state=1)
    svm.fit(q_train_x,train_y)
    predictions=svm.predict(q_val_x)
    acc=accuracy_score(val_y, predictions)
    if acc > best_acc:
      best_acc=acc
      best_c=c
      best_gamma=gamma

#print("Quant SVM best acc = ", best_acc)
#print("Quant SVM best c = ", best_c)
#print("Quant SVM best acc = ", best_gamma)

#print(best_acc,best_n_estimators,best_criterion)
svm = SVC(C=best_c,gamma = best_gamma,random_state=1)
svm.fit(q_train_val_x,train_val_y)
predictions=svm.predict(q_test_x)
acc=accuracy_score(test_y, predictions)
print(f'SVMQuantization Accuracy: {acc}')


#Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
min_samples_split_list=range(2, 10)
max_features_list = [1,2,4,6,8,10,'auto','sqrt','log2']
criterion_list=["gini", "entropy"]


#for binarization part
best_acc = -np.inf

for min_samples in min_samples_split_list:
  for max_features in max_features_list:
    for criterion in criterion_list:
      dt = DecisionTreeClassifier(criterion = criterion,min_samples_split = min_samples, max_features = max_features, random_state=1)
      dt.fit(b_train_x,train_y)
      predictions=dt.predict(b_val_x)
      acc=accuracy_score(val_y, predictions)
      if acc > best_acc:
        best_acc=acc
        best_criterion = criterion
        best_min_samples=min_samples
        best_max_features=max_features

#print("Binarization dt best acc = ", best_acc)
#print("Binarization dt best criterion = ", best_criterion)
#print("Binarization dt best min_samples = ", best_min_samples)
#print("Binarization dt best min_samples = ", best_max_features)

dt = DecisionTreeClassifier(criterion = criterion,min_samples_split = min_samples, max_features = max_features, random_state=1)
dt.fit(b_train_val_x,train_val_y)
predictions=dt.predict(b_test_x)
acc=accuracy_score(test_y, predictions)
print(f'DecisionTreeBinarization Accuracy: {acc}')

#for quantization part
best_acc = -np.inf

for min_samples in min_samples_split_list:
  for max_features in max_features_list:
    for criterion in criterion_list:
      dt=DecisionTreeClassifier(criterion = criterion,min_samples_split = min_samples, max_features = max_features, random_state=1)
      dt.fit(q_train_x,train_y)
      predictions=dt.predict(q_val_x)
      acc=accuracy_score(val_y, predictions)
      if acc > best_acc:
        best_acc=acc
        best_criterion = criterion
        best_min_samples=min_samples
      best_max_features=max_features

#print("Quant dt best acc = ", best_acc)
#print("Quant dt best criterion = ", best_criterion)
#print("Quant dt best min_samples = ", best_min_samples)
#print("Quant dt best min_samples = ", best_max_features)

#print(best_acc,best_n_estimators,best_criterion)
dt = DecisionTreeClassifier(criterion = criterion,min_samples_split = min_samples, max_features = max_features, random_state=1)
dt.fit(q_train_val_x,train_val_y)
predictions=dt.predict(q_test_x)
acc=accuracy_score(test_y, predictions)
print(f'DecisionTreeQuantization Accuracy: {acc}')